In [1]:
import numpy as np
import torch 
import time
from torch import nn
from torchvision import datasets, transforms,utils
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim


transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5],std=[0.5])])


train_dataset = datasets.FashionMNIST(root='./data', 
                                            train=True, 
                                            transform=transform, 
                                            download=True)
test_dataset = datasets.FashionMNIST(root='./data', 
                                            train=False, 
                                            transform=transform, 
                                            download=True)

In [2]:
batch_size = 128
shuffle = True
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=True, # 装载过程中随机乱序
                                            num_workers=1) # 表示2个子进程加载数据
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=False,
                                            num_workers=1) 

In [3]:
next(iter(test_dataset))

(tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.000

In [4]:
num_inputs = 28*28
num_hidens = 256
num_outputs = 10

w1 = torch.tensor(np.random.normal(0,0.01,[num_inputs, num_hidens]), dtype=torch.float32, requires_grad=True)
b1 = torch.zeros([num_hidens], requires_grad=True)

w2 = torch.tensor(np.random.normal(0, 0.01, [num_hidens, num_outputs]), dtype=torch.float32, requires_grad=True)
b2 = torch.zeros([num_outputs], requires_grad=True)


paramls = [w1, w2, b1, b2]


def Rule(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

loss = nn.CrossEntropyLoss() # y为分类标签index，y_hat为预测概率


def acc(y_hat, y):
    _, pre_labels = torch.max(y_hat, dim=1)
    acc_count = (pre_labels == y).sum().item()
    return acc_count

def sgd(paramls, lr):
    for param in paramls:
        param.data -= lr*param.grad
        param.grad.zero_()

def net(X):
    X = X.reshape(-1, num_inputs)
    X = torch.matmul(X, w1) + b1
    X = Rule(X)
    X = torch.matmul(X, w2) + b2
    return X 

In [5]:
epoch = 5
lr = 0.1
for i in range(epoch):
    l_trian, acc_count, count = 0, 0, 0
    for n, (x, y) in enumerate(train_loader):
        y_hat = net(x)
        l = loss(y_hat, y)
        l.backward()
        sgd(paramls, lr)
        acc_count += acc(y_hat, y)
        count += y.size(0) 
    print('epoch:', i, "acc", acc_count/count)
    print(count, acc_count)


epoch: 0 acc 0.76665
60000 45999
epoch: 1 acc 0.8395666666666667
60000 50374
epoch: 2 acc 0.857
60000 51420
epoch: 3 acc 0.8677833333333334
60000 52067
epoch: 4 acc 0.8739666666666667
60000 52438
